In [6]:
import numpy as np

In [7]:
# state transition matrix
A = np.array([[0.1, 0.5, 0.4],
              [0.6, 0.3, 0.1],
              [0.3, 0.6, 0.1]])

# state-output association probability matrix
B = np.array([[0.3, 0.4, 0.3],
              [0.4, 0.2, 0.4],
              [0.6, 0.2, 0.2]])

# Initial probability vector
P = np.array([0.3, 0.4, 0.3])

# Observable state vector
O = [1, 2, 0]

In [8]:
def forward_var(A, B, P, O):
    T = len(O)
    N = A.shape[0]
    alpha = np.zeros((T, N))
    for t in range(alpha.shape[0]):
        for i in range(alpha.shape[1]):
            if t == 0:
                alpha[t, i] = B[i, O[0]]*P[i]
            else:
                sum1 = 0
                for k in range(A.shape[0]):
                    sum1 += alpha[t-1, k]*A[k, i]
                alpha[t,i] = sum1*B[i, O[t]]
    return (alpha)

In [10]:
alpha = forward_var(A, B, P, O)
print(alpha)

[[0.12     0.08     0.06    ]
 [0.0234   0.048    0.0124  ]
 [0.010458 0.013416 0.00924 ]]


In [11]:
def backward_var(A, B, P, O):
    T = len(O)
    N = A.shape[0]
    beta = np.zeros((T, N))
    for t in range(beta.shape[0]):
        for i in range(beta.shape[1]):
            if t == beta.shape[0]-1:
                beta[t, i] = 1
            else:
                sum1 = 0
                for j in range(A.shape[0]):
                    sum1 += A[i, j]*B[j, O[t+1]]*beta[t+1, j]
                beta[t,i] = sum1
    return (beta)  

In [12]:
beta = backward_var(A, B, P, O)
print(beta)

[[0. 0. 0.]
 [0. 0. 0.]
 [1. 1. 1.]]


In [13]:
def forward_eval(alpha):
    prob = np.sum(alpha[-1, :])
    return prob

In [15]:
forward_prob = forward_eval(alpha)
print(forward_prob)

0.033114000000000005


In [18]:
def backward_eval(B, beta, O):
    vec1 = np.ravel(B[:, O[0]])
    vec2 = np.ravel(beta[0, :])
    prob = np.dot(vec1, vec2)
    return prob

In [20]:
backward_prob = backward_eval(B, beta, O)
print(backward_prob)

0.0


In [21]:
def gammaeval(alpha, beta):
    gamma = np.multiply(alpha, beta)
    statewise_sum = np.sum(gamma, axis = 1)
    statewise_sum = statewise_sum.reshape(len(statewise_sum), 1)
    gamma = gamma/statewise_sum
    return gamma

In [22]:
gamma = gammaeval(alpha, beta)
print(gamma)

[[       nan        nan        nan]
 [       nan        nan        nan]
 [0.31581808 0.40514586 0.27903606]]


c:\users\keshav anand\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [23]:
def viterbi(A, B, P, O):
    T = len(O)
    N = A.shape[0]
    delta = np.zeros((T, N))
    psi = np.zeros((T, N))
    optim_state = []
    for t in range(delta.shape[0]):
        for i in range(delta.shape[1]):
            if t == 0:
                delta[t, i] = B[i, O[0]]*P[i]
            else:
                ls = []
                for k in range(N):
                    val = delta[t-1, k]*A[k, i]*B[i, O[t]]
                    ls.append(val)
                arr = np.array(ls)
                max_val = np.max(arr)
                max_val_idx = np.argmax(arr)
                delta[t, i] = max_val
                psi[t, i] = max_val_idx
    final_state_val = np.ravel(delta[-1, :])
    idx = np.argmax(final_state_val)
    optim_state.append(idx)
    for t in range(T-2, -1, -1):
        idx = psi[t, idx]
        idx = int(idx)
        optim_state.append(idx)
        
    optim_state = optim_state[::-1]
    return optim_state
    

In [24]:
optim_state = viterbi(A, B, P, O)
print(optim_state)

[0, 1, 0]


In [25]:
def joint_state_measure(A, B, alpha, beta, O):
    T = len(O)
    N = A.shape[0]
    eta = np.zeros((T-1, N, N))
    for t in range(eta.shape[0]):
        sum1 = 0
        for i in range(eta.shape[1]):
            for j in range(eta.shape[2]):
                eta[t, i, j] = alpha[t, i]*A[i, j]*B[j, O[t+1]]*beta[t+1, j]
                sum1 = eta[t, i, j]
    eta[t, :, :] = eta[t, :, :]/sum1
    return eta

In [26]:
eta = joint_state_measure(A, B, alpha, beta, O)
print(eta)

[[[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.94354839  6.29032258  7.5483871 ]
  [11.61290323  7.74193548  3.87096774]
  [ 1.5         4.          1.        ]]]


In [29]:
def baum_welch(eta, gamma, A, B, P, O):
    N = A.shape[0]
    M = B.shape[1]
    mod_A = np.zeros((N, N))
    mod_B = np.zeros((N, M))
    mod_P = np.zeros(N)
    
    for i in range(len(mod_P)):
        mod_P[i] = gamma[0, i] 
        
    for i in range(mod_A.shape[0]):
        for j in range(mod_A.shape[1]):
            num_sum = 0
            denom_sum = 0
            for t in range(eta.shape[0]):
                num_sum += eta[t, i, j]
                denom_sum += gamma[t, i]
            a = num_sum/denom_sum
            mod_A[i, j] = a
            
    for j in range(B.shape[0]):
        for k in range(B.shape[1]):
            num_sum = 0
            denom_sum = 0
            for t in range(gamma.shape[0]):
                if O[t] == k:
                    num_sum += gamma[t, j]
                    denom_sum += gamma[t, i]
            b = num_sum/denom_sum
            mod_B[j, k] = b

In [32]:
baum_welch(eta, gamma, A, B, P, O)